In [20]:
import pandas as pd
import numpy as np

import itertools
import multiprocessing as mp

import findspark
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext

import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")

import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.types import StructType, StructField, NumericType

#READ FILE FOR FORMATTING (PANDAS)
panda_df = pd.read_csv(r'C:\Users\Cayo\Downloads\s5117135_Project_Code\s5117135_Project_Code\BOUN_TCP_Anon.csv') #File path
panda_df



,Time,Frame_No,Frame_lng,Src_ip,Dst_IP,Src_Port,Dst_Port,SYN,ACK,RST,TTL,Pro
0,0.000000,1,64,10.50.197.71,10.50.192.199,49543.0,443.0,Not set,Set,Not set,127,TCP
1,0.000002,2,2978,10.50.192.199,10.50.197.71,443.0,49543.0,Not set,Set,Not set,127,TCP
2,0.000040,3,64,10.50.197.71,10.50.192.199,49543.0,443.0,Not set,Set,Not set,127,TCP
3,0.000042,4,4438,10.50.192.199,10.50.197.71,443.0,49543.0,Not set,Set,Not set,127,TCP
4,0.000053,5,64,10.50.197.71,10.50.192.199,49543.0,443.0,Not set,Set,Not set,127,TCP
...,...,...,...,...,...,...,...,...,...,...,...,...
9335600,482.662762,9335601,1396,10.50.197.71,216.58.208.111,NaN,NaN,NaN,NaN,NaN,127,UDP
9335601,482.662764,9335602,1396,10.50.197.71,216.58.208.111,NaN,NaN,NaN,NaN,NaN,127,UDP
9335602,482.662912,9335603,1396,10.50.197.71,216.58.208.111,NaN,NaN,NaN,NaN,NaN,127,UDP
9335603,482.662913,9335604,1396,10.50.197.71,216.58.208.111,NaN,NaN,NaN,NaN,NaN,127,UDP


In [21]:
#Filtering attack window because of hardware constraints
panda_df1 = panda_df[panda_df['Time'].values < 403]
panda_df2 = panda_df1[panda_df1['Time'].values > 379]
panda_df2

,Time,Frame_No,Frame_lng,Src_ip,Dst_IP,Src_Port,Dst_Port,SYN,ACK,RST,TTL,Pro
7417803,379.000005,7417804,87,10.50.197.37,64.15.113.178,NaN,NaN,NaN,NaN,NaN,127,UDP
7417804,379.000035,7417805,64,10.50.211.93,69.16.175.42,56470.0,80.0,Not set,Set,Not set,126,TCP
7417805,379.000180,7417806,64,192.168.104.66,64.15.113.145,49499.0,443.0,Not set,Set,Not set,128,TCP
7417806,379.000249,7417807,427,192.168.72.22,10.50.13.91,58658.0,80.0,Not set,Set,Not set,"63,64",TCP
7417807,379.000251,7417808,1500,192.168.64.93,54.243.43.208,49829.0,443.0,Not set,Set,Not set,"63,64",TCP
...,...,...,...,...,...,...,...,...,...,...,...,...
7895619,402.999840,7895620,1500,23.0.174.48,192.168.74.64,80.0,52241.0,Not set,Set,Not set,"252,57",TCP
7895620,402.999841,7895621,1500,23.0.174.48,192.168.74.64,80.0,52241.0,Not set,Set,Not set,"252,57",TCP
7895621,402.999843,7895622,1500,23.0.174.48,192.168.74.64,80.0,52241.0,Not set,Set,Not set,"252,57",TCP
7895622,402.999843,7895623,1518,64.15.113.145,192.168.104.66,443.0,49499.0,Not set,Set,Not set,56,TCP


In [22]:
#Filtering the attack window
panda_df = panda_df.drop(labels=range(7895623, 9335604), axis=0)
panda_df = panda_df.drop(labels=range(0, 7417803), axis=0)
# panda_df
# panda_df1 = panda_df[panda_df['Time'].values < 104]
panda_df

,Time,Frame_No,Frame_lng,Src_ip,Dst_IP,Src_Port,Dst_Port,SYN,ACK,RST,TTL,Pro
7417803,379.000005,7417804,87,10.50.197.37,64.15.113.178,NaN,NaN,NaN,NaN,NaN,127,UDP
7417804,379.000035,7417805,64,10.50.211.93,69.16.175.42,56470.0,80.0,Not set,Set,Not set,126,TCP
7417805,379.000180,7417806,64,192.168.104.66,64.15.113.145,49499.0,443.0,Not set,Set,Not set,128,TCP
7417806,379.000249,7417807,427,192.168.72.22,10.50.13.91,58658.0,80.0,Not set,Set,Not set,"63,64",TCP
7417807,379.000251,7417808,1500,192.168.64.93,54.243.43.208,49829.0,443.0,Not set,Set,Not set,"63,64",TCP
...,...,...,...,...,...,...,...,...,...,...,...,...
7895619,402.999840,7895620,1500,23.0.174.48,192.168.74.64,80.0,52241.0,Not set,Set,Not set,"252,57",TCP
7895620,402.999841,7895621,1500,23.0.174.48,192.168.74.64,80.0,52241.0,Not set,Set,Not set,"252,57",TCP
7895621,402.999843,7895622,1500,23.0.174.48,192.168.74.64,80.0,52241.0,Not set,Set,Not set,"252,57",TCP
7895622,402.999843,7895623,1518,64.15.113.145,192.168.104.66,443.0,49499.0,Not set,Set,Not set,56,TCP


In [24]:
#FORMAT DATA

# #Replace all "Not Set" to 0 in SYN, ACK and RST columns
panda_df['SYN'] = panda_df['SYN'].replace(['Not set'], 0)
panda_df['ACK'] = panda_df['ACK'].replace(['Not set'], 0)
panda_df['RST'] = panda_df['RST'].replace(['Not set'], 0)


# #Replace all "Set" to 1 in SYN, ACK and RST columns
panda_df['SYN'] = panda_df['SYN'].replace(['Set'], 1)
panda_df['ACK'] = panda_df['ACK'].replace(['Set'], 1)
panda_df['RST'] = panda_df['RST'].replace(['Set'], 1)


#Remove ALL rows with NaN data
panda_df.dropna(subset = ["SYN"], inplace=True)
panda_df.dropna(subset = ["ACK"], inplace=True)
panda_df.dropna(subset = ["RST"], inplace=True)
panda_df.dropna(subset = ["Src_ip"], inplace=True)
panda_df.dropna(subset = ["Dst_IP"], inplace=True)
panda_df.dropna(subset = ["Frame_lng"], inplace=True)

#Change Src_Port, Dst_Port, SYN, ACK, and RST from FLOAT to INT
panda_df['Src_Port'] = panda_df['Src_Port'].astype(int)
panda_df['Dst_Port'] = panda_df['Dst_Port'].astype(int)
panda_df['SYN'] = panda_df['SYN'].astype(int)
panda_df['ACK'] = panda_df['ACK'].astype(int)
panda_df['RST'] = panda_df['RST'].astype(int)

#Change time to per second (0 - 482)
panda_df["Time"] = panda_df["Time"].astype(np.uint32)

#Print panda_df to screen
panda_df

,Time,Frame_No,Frame_lng,Src_ip,Dst_IP,Src_Port,Dst_Port,SYN,ACK,RST,TTL,Pro
7417804,379,7417805,64,10.50.211.93,69.16.175.42,56470,80,0,1,0,126,TCP
7417805,379,7417806,64,192.168.104.66,64.15.113.145,49499,443,0,1,0,128,TCP
7417806,379,7417807,427,192.168.72.22,10.50.13.91,58658,80,0,1,0,"63,64",TCP
7417807,379,7417808,1500,192.168.64.93,54.243.43.208,49829,443,0,1,0,"63,64",TCP
7417809,379,7417810,64,10.50.211.93,69.16.175.42,56470,80,0,1,0,126,TCP
...,...,...,...,...,...,...,...,...,...,...,...,...
7895618,402,7895619,1500,23.0.174.48,192.168.74.64,80,52241,0,1,0,"252,57",TCP
7895619,402,7895620,1500,23.0.174.48,192.168.74.64,80,52241,0,1,0,"252,57",TCP
7895620,402,7895621,1500,23.0.174.48,192.168.74.64,80,52241,0,1,0,"252,57",TCP
7895621,402,7895622,1500,23.0.174.48,192.168.74.64,80,52241,0,1,0,"252,57",TCP


In [25]:
#Output formatted panda_df as new CSV ready for Spark
panda_df.to_csv(r'C:\Users\Cayo\Downloads\s5117135_Project_Code\s5117135_Project_Code\BOUN_TCP_Format.csv', index=False)